In [6]:
import tracemalloc
import linecache
tracemalloc.start()

In [10]:
a = [i for i in range(100)]
snapshot1 = tracemalloc.take_snapshot()

# ... call the function leaking memory ...
d = [i**i for i in range(1000)]
snapshot2 = tracemalloc.take_snapshot()

top_stats = snapshot2.compare_to(snapshot1, 'lineno')

print("[ Top 10 differences ]")
for stat in top_stats[:10]:
    print(stat)

[ Top 10 differences ]
C:\Users\Santiago_Ortiz\AppData\Local\Temp\ipykernel_1372\1992036833.py:5: size=637 KiB (+637 KiB), count=997 (+997), average=655 B
C:\Users\Santiago_Ortiz\.pyenv\pyenv-win\versions\3.12.0\Lib\tracemalloc.py:558: size=153 KiB (+44.2 KiB), count=2893 (+916), average=54 B
C:\Users\Santiago_Ortiz\.pyenv\pyenv-win\versions\3.12.0\Lib\tokenize.py:537: size=75.1 KiB (-33.1 KiB), count=1372 (-606), average=56 B
C:\Users\Santiago_Ortiz\.pyenv\pyenv-win\versions\3.12.0\Lib\tracemalloc.py:193: size=41.2 KiB (-21.4 KiB), count=879 (-456), average=48 B
c:\Users\Santiago_Ortiz\mastering-python\.venv\Lib\site-packages\IPython\utils\_process_win32.py:174: size=248 B (-2741 B), count=3 (-16), average=83 B
C:\Users\Santiago_Ortiz\.pyenv\pyenv-win\versions\3.12.0\Lib\collections\__init__.py:508: size=0 B (-1747 B), count=0 (-9)
C:\Users\Santiago_Ortiz\.pyenv\pyenv-win\versions\3.12.0\Lib\json\decoder.py:353: size=8507 B (-1234 B), count=67 (-22), average=127 B
C:\Users\Santiago_Or

In [11]:
def display_top(snapshot, key_type='lineno', limit=10):
    snapshot = snapshot.filter_traces((
        tracemalloc.Filter(False, "<frozen importlib._bootstrap>"),
        tracemalloc.Filter(False, "<unknown>"),
    ))
    top_stats = snapshot.statistics(key_type)

    print("Top %s lines" % limit)
    for index, stat in enumerate(top_stats[:limit], 1):
        frame = stat.traceback[0]
        print("#%s: %s:%s: %.1f KiB"
              % (index, frame.filename, frame.lineno, stat.size / 1024))
        line = linecache.getline(frame.filename, frame.lineno).strip()
        if line:
            print('    %s' % line)

    other = top_stats[limit:]
    if other:
        size = sum(stat.size for stat in other)
        print("%s other: %.1f KiB" % (len(other), size / 1024))
    total = sum(stat.size for stat in top_stats)
    print("Total allocated size: %.1f KiB" % (total / 1024))

In [12]:
display_top(snapshot2)

Top 10 lines
#1: C:\Users\Santiago_Ortiz\AppData\Local\Temp\ipykernel_1372\1992036833.py:5: 637.4 KiB
    d = [i**i for i in range(1000)]
#2: C:\Users\Santiago_Ortiz\.pyenv\pyenv-win\versions\3.12.0\Lib\linecache.py:137: 241.3 KiB
    lines = fp.readlines()
#3: <string>:1: 166.6 KiB
#4: C:\Users\Santiago_Ortiz\.pyenv\pyenv-win\versions\3.12.0\Lib\tracemalloc.py:558: 153.4 KiB
    traces = _get_traces()
#5: c:\Users\Santiago_Ortiz\mastering-python\.venv\Lib\site-packages\IPython\core\compilerop.py:174: 81.8 KiB
    [line + "\n" for line in transformed_code.splitlines()],
#6: C:\Users\Santiago_Ortiz\.pyenv\pyenv-win\versions\3.12.0\Lib\tokenize.py:537: 75.1 KiB
    for info in it:
#7: C:\Users\Santiago_Ortiz\.pyenv\pyenv-win\versions\3.12.0\Lib\ast.py:52: 63.8 KiB
    return compile(source, filename, mode, flags,
#8: C:\Users\Santiago_Ortiz\.pyenv\pyenv-win\versions\3.12.0\Lib\tracemalloc.py:469: 51.8 KiB
    new_traces = [trace for trace in self.traces._traces
#9: C:\Users\Santiago_Orti

In [14]:
snapshot2.statistics('lineno')

[<Statistic traceback=<Traceback (<Frame filename='C:\\Users\\Santiago_Ortiz\\AppData\\Local\\Temp\\ipykernel_1372\\1992036833.py' lineno=5>,)> size=652700 count=997>,
 <Statistic traceback=<Traceback (<Frame filename='C:\\Users\\Santiago_Ortiz\\.pyenv\\pyenv-win\\versions\\3.12.0\\Lib\\linecache.py' lineno=137>,)> size=247128 count=2705>,
 <Statistic traceback=<Traceback (<Frame filename='<string>' lineno=1>,)> size=170568 count=1580>,
 <Statistic traceback=<Traceback (<Frame filename='C:\\Users\\Santiago_Ortiz\\.pyenv\\pyenv-win\\versions\\3.12.0\\Lib\\tracemalloc.py' lineno=558>,)> size=157128 count=2893>,
 <Statistic traceback=<Traceback (<Frame filename='c:\\Users\\Santiago_Ortiz\\mastering-python\\.venv\\Lib\\site-packages\\IPython\\core\\compilerop.py' lineno=174>,)> size=83751 count=839>,
 <Statistic traceback=<Traceback (<Frame filename='C:\\Users\\Santiago_Ortiz\\.pyenv\\pyenv-win\\versions\\3.12.0\\Lib\\tokenize.py' lineno=537>,)> size=76856 count=1372>,
 <Statistic tracebac

In [16]:
first_size, first_peak = tracemalloc.get_traced_memory()
first_size, first_peak

(2236395, 982754876)

In [17]:
# Example code: compute a sum with a large temporary list
large_sum = sum(list(range(100000)))

first_size, first_peak = tracemalloc.get_traced_memory()

tracemalloc.reset_peak()

# Example code: compute a sum with a small temporary list
small_sum = sum(list(range(1000)))

second_size, second_peak = tracemalloc.get_traced_memory()

print(f"{first_size=}, {first_peak=}")
print(f"{second_size=}, {second_peak=}")

first_size=2253601, first_peak=982754876
second_size=2253741, second_peak=2285601


Using reset_peak() ensured we could accurately record the peak during the computation of small_sum, even though it is much smaller than the overall peak size of memory blocks since the start() call. Without the call to reset_peak(), second_peak would still be the peak from the computation large_sum (that is, equal to first_peak). In this case, both peaks are much higher than the final memory usage, and which suggests we could optimise (by removing the unnecessary call to list, and writing sum(range(...))).